In [41]:
import pandas as pd 
import numpy as np

In [42]:
movies = pd.read_csv('data/data_marvel.csv')

In [43]:
sales_2008 = pd.read_csv('data/sales_2008.csv')
sales_2009 = pd.read_csv('data/sales_2009.csv')
sales_2010 = pd.read_csv('data/sales_2010.csv')
sales_2011 = pd.read_csv('data/sales_2011.csv')
sales_2012 = pd.read_csv('data/sales_2012.csv')
sales_2013 = pd.read_csv('data/sales_2013.csv')
sales_2014 = pd.read_csv('data/sales_2014.csv')
sales_2015 = pd.read_csv('data/sales_2015.csv')
sales_2016 = pd.read_csv('data/sales_2016.csv')
sales_2017 = pd.read_csv('data/sales_2017.csv')
sales_2018 = pd.read_csv('data/sales_2018.csv')
sales_2019 = pd.read_csv('data/sales_2019.csv')
sales_2020 = pd.read_csv('data/sales_2020.csv')
sales_2021 = pd.read_csv('data/sales_2021.csv')

In [44]:
def ccleaner(data, date):
    data['Publish Date'] = date
    second_column = data.pop('Publish Date')
    data.insert(1, 'Publish Date', second_column)
    data['Price'] = data['Price'].replace('[\$,]', '', regex=True).astype(float)
    data['Est. units'] = data['Est. units'].replace(',', '', regex=True).astype(float)
    data['Total'] = data['Price'] * data['Est. units']
    data['Comic_ID'] = data.index
    
    data_list = []
    df = pd.DataFrame(columns=['Comic_ID', 'Publish Date', 'Comic-book Title', 'Price', 'Publisher', 'Est. units', 'Total'])
    
    for index, result in data.iterrows():
        local = {}
        local['Comic_ID'] = result['Comic_ID']
        local['Publish Date'] = result['Publish Date']
        local['Comic-book Title'] = result['Comic-book Title']
        local['Price'] = result['Price']
        local['Publisher'] = result['Publisher']
        local['Est. units'] = result['Est. units']
        local['Total'] = result['Total']
        data_list.append(local)
        
    df = pd.DataFrame(data_list)
    file_name = f'comic_{date}.csv'
    df.to_csv(f'data/{file_name}', index=False)   
    return df

In [45]:
def creator(data, check_list, date):
    matches = data[data['Comic-book Title'].str.contains('|'.join(check_list))]
    result_dict = {}
    for valor_b in check_list:
        matching_rows = matches[matches['Comic-book Title'].str.contains(valor_b)]
        if not matching_rows.empty:
            result_dict[valor_b] = {'Name': valor_b, 'Total': matching_rows['Total'].sum()} 
    result_df = pd.DataFrame.from_dict(result_dict, orient='index').reset_index()
    result_df.columns = ['Character_Name', 'Character_ID', 'Total']
    result_df['Character_ID'] = result_df.index
    result_df['Date'] = date
    result_df = result_df[['Character_ID', 'Date', 'Character_Name', 'Total']]
    file_name = f'character_{date}.csv'
    result_df.to_csv(f'data/{file_name}', index=False)
    return result_df

In [46]:
def save_csv(data, name):
    file_name = f'{name}.csv'
    data.to_csv(f'data/{file_name}', index=False)

In [47]:
def add_date(data):
    data['Release year'] = data['Release date(United States)'].str.extract(r'(\d{4})')
    data['Release year'] = data['Release year'].astype(int)
    data.drop('Release date(United States)', axis=1, inplace=True)
    second_column = data.pop('Release year')
    data.insert(2, 'Release year', second_column)
    return data

In [48]:
def add_index(data, name):
    data[name] = data.index
    first_column = data.pop(name)
    data.insert(0, name, first_column)
    return data

In [49]:
check_list = [
    'Absorbing Man', 'A.I.M.', 'Anole', 'Apocalypse', 'Archangel', 'Avalanche', 'Bastion', 'Beast',
    'Beyonder', 'Black Panther', 'Black Widow', 'Blob', 'Cable', 'Cannonball', 'Captain America',
    'Cassandra Nova', 'Chamber', 'Colossus', 'Crossbones', 'Cypher', 'Daredevil', 'Dark Phoenix',
    'Deadpool', 'Doctor Strange', 'Domino', 'Dust', 'Emma Frost', 'Exodus', 'Falcon', 'Fantomex',
    'Feral', 'Forge', 'Gambit', 'Gideon', 'Grandmaster', 'Grim Reaper', 'Havok', 'Hela', 'Hellion',
    'Hope Summers', 'Hulk', 'Iceman', 'Iron Man', 'Iron Patriot', 'Jean Grey', 'Jean Grey', 'Jubilee',
    'Juggernaut', 'Juggernaut', 'Kang', 'Karma', 'Korvac', 'Klaw', 'Lady Deathstrike', 'Living Laser',
    'Loki', 'Longshot', 'Luke Cage', 'M.O.D.O.K.', 'Magneto', 'Marrow', 'Mastermind', 'Master Mold',
    'Marauders', 'Marvel Boy', 'Multiple Man', 'Mystique', 'Nightcrawler', 'Omega Red', 'Onslaught',
    'Pixie', 'Polaris', 'Prodigy', 'Professor X', 'Psylocke', 'Red Skull', 'Rhino', 'Rockslide',
    'Rogue', 'Scarlet Witch', 'Sebastian Shaw', 'Selene', 'Shadowcat', 'Shadow King', 'Shatterstar',
    'Silver Samurai', 'Sinister Six', 'Skrulls', 'Sage', 'Spider-Man', 'Storm', 'Strong Guy', 'Sunspot',
    'Surge', 'Taskmaster', 'Thanos', 'The Hand', 'The Hood', 'Thor', 'U-Foes', 'Ultron', 'Vision',
    'Warpath', 'Wasp', 'William Stryker', 'Wolverine', 'Wolverine', 'X-23','X-Men', 'Avengers',
    'Ghost Rider', 'Venom', 'Dark Phoenix', 'Inhumans', 'Guardians of the Galaxy', 'Logan', 'Fantastic Four',
    'Punisher'
]




In [50]:
df_08 = ccleaner(sales_2008,'2008')
df_09 = ccleaner(sales_2009, '2009')
df_10 = ccleaner(sales_2010, '2010')
df_11 = ccleaner(sales_2011, '2011')
df_12 = ccleaner(sales_2012, '2012')
df_13 = ccleaner(sales_2013, '2013')
df_14 = ccleaner(sales_2014, '2014')
df_15 = ccleaner(sales_2015, '2015')
df_16 = ccleaner(sales_2016, '2016')
df_17 = ccleaner(sales_2017, '2017')
df_18 = ccleaner(sales_2018, '2018')
df_19 = ccleaner(sales_2019, '2019')
df_20 = ccleaner(sales_2020, '2020')
df_21 = ccleaner(sales_2021, '2021')

In [51]:
creator(df_08, check_list, '2008')
creator(df_09, check_list, '2009')
creator(df_10, check_list, '2010')
creator(df_11, check_list, '2011')
creator(df_12, check_list, '2012')
creator(df_13, check_list, '2013')
creator(df_14, check_list, '2014')
creator(df_15, check_list, '2015')
creator(df_16, check_list, '2016')
creator(df_17, check_list, '2017')
creator(df_18, check_list, '2018')
creator(df_19, check_list, '2019')
creator(df_20, check_list, '2020')
creator(df_21, check_list, '2021')

,Character_ID,Date,Character_Name,Total
0,0,2021,Black Panther,1175795.88
1,1,2021,Black Widow,797640.90
2,2,2021,Cable,740334.43
3,3,2021,Captain America,1989043.71
4,4,2021,Daredevil,2132739.68
5,5,2021,Deadpool,883259.87
6,6,2021,Doctor Strange,581546.78
7,7,2021,Hellion,1091352.78
8,8,2021,Hulk,3230742.07
9,9,2021,Iron Man,1317025.27


In [52]:
add_date(movies)

,Title,Distributor(s),Release year,Budget (millions),Opening weekend(North America),North America,Other territories,Worldwide
0,Iron Man,Paramount Pictures,2008,$140,$98618668,$318412101,$266762121,$585174222
1,The Incredible Hulk,Universal Pictures,2008,$150,$55414050,$134806913,$128620638,$263427551
2,Punisher: War Zone,Lionsgate Films,2008,$35,$4271451,$8050977,$2049059,$10100036
3,X-Men Origins: Wolverine,20th Century Fox,2009,$150,$85058003,$179883157,$193179707,$373062864
4,Iron Man 2,Paramount Pictures,2010,$200,$128122480,$312433331,$311500000,$623933331
5,Thor,Paramount Pictures,2011,$150,$65723338,$181030624,$268295994,$449326618
6,X-Men: First Class,20th Century Fox,2011,$160,$55101604,$146408305,$207215819,$353624124
7,Captain America: The First Avenger,Paramount Pictures,2011,$140,$65058524,$176654505,$193915269,$370569774
8,Ghost Rider: Spirit of Vengeance,Sony Pictures,2012,$57,$22115334,$51774002,$80789928,$132563930
9,The Avengers,Walt Disney Studios Motion Pictures,2012,$220,$207438708,$623357910,$895455078,$1518812988


In [53]:
add_index(movies, 'Movie_ID')

,Movie_ID,Title,Distributor(s),Release year,Budget (millions),Opening weekend(North America),North America,Other territories,Worldwide
0,0,Iron Man,Paramount Pictures,2008,$140,$98618668,$318412101,$266762121,$585174222
1,1,The Incredible Hulk,Universal Pictures,2008,$150,$55414050,$134806913,$128620638,$263427551
2,2,Punisher: War Zone,Lionsgate Films,2008,$35,$4271451,$8050977,$2049059,$10100036
3,3,X-Men Origins: Wolverine,20th Century Fox,2009,$150,$85058003,$179883157,$193179707,$373062864
4,4,Iron Man 2,Paramount Pictures,2010,$200,$128122480,$312433331,$311500000,$623933331
5,5,Thor,Paramount Pictures,2011,$150,$65723338,$181030624,$268295994,$449326618
6,6,X-Men: First Class,20th Century Fox,2011,$160,$55101604,$146408305,$207215819,$353624124
7,7,Captain America: The First Avenger,Paramount Pictures,2011,$140,$65058524,$176654505,$193915269,$370569774
8,8,Ghost Rider: Spirit of Vengeance,Sony Pictures,2012,$57,$22115334,$51774002,$80789928,$132563930
9,9,The Avengers,Walt Disney Studios Motion Pictures,2012,$220,$207438708,$623357910,$895455078,$1518812988


In [73]:
digite = []
years =movies['Release year'].unique()
for year in years:
    last_two = year % 100
    digite.append(last_two)
digite    

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [75]:
check_letter = {'c':3,'d':4,
                'h':8,'m':13
                }
check = 'c'
for k, v in check_letter.items():
    if check in k:
        letter = v
letter        

3

In [ ]:
def add_index(data, name):
    data[name] = data.index
    first_column = data.pop(name)
    data.insert(0, name, first_column)
    return data

In [68]:
save_csv(movies, 'movie box office')